In [1]:
import re
import get_xml_etree
import pandas as pd
import os

### Para 1 XML

In [2]:
name_file = 'ag_cost_IRR_MGMT'
path = 'C:/Users/ignacio.delatorre/Documents/Understanding Gcam/gcam-v8.2-Windows-Release-Package/input/gcamdata/xml/'
file = path + name_file + '.xml'

Ver su jerarquía

In [5]:
get_xml_etree._1_tags_hierarchy(file)

NameError: name 'get_xml_etree' is not defined

Entender los tags asociados

In [4]:
get_xml_etree._1_print_attr (file)

region: ['name']
AgSupplySector: ['name']
AgSupplySubsector: ['name']
AgProductionTechnology: ['name']
period: ['year']
CO2: ['name']


Sacar los nombres de las variables asociadas a cada región

In [5]:
region_variables = get_xml_etree._1_getVarnames (file)
region_variables

{'USA': ['Corn',
  'CornC4_NelsonR',
  'CornC4_NelsonR_IRR_hi',
  'CornC4_NelsonR_IRR_lo',
  'CornC4_NelsonR_RFD_hi',
  'CornC4_NelsonR_RFD_lo',
  'CornC4_MexCstNW',
  'CornC4_MexCstNW_IRR_hi',
  'CornC4_MexCstNW_IRR_lo',
  'CornC4_MexCstNW_RFD_hi',
  'CornC4_MexCstNW_RFD_lo',
  'CornC4_Caribbean',
  'CornC4_Caribbean_IRR_hi',
  'CornC4_Caribbean_IRR_lo',
  'CornC4_Caribbean_RFD_hi',
  'CornC4_Caribbean_RFD_lo',
  'CornC4_California',
  'CornC4_California_IRR_hi',
  'CornC4_California_IRR_lo',
  'CornC4_California_RFD_hi',
  'CornC4_California_RFD_lo',
  'CornC4_MissppRN',
  'CornC4_MissppRN_IRR_hi',
  'CornC4_MissppRN_IRR_lo',
  'CornC4_MissppRN_RFD_hi',
  'CornC4_MissppRN_RFD_lo',
  'CornC4_MissppRS',
  'CornC4_MissppRS_IRR_hi',
  'CornC4_MissppRS_IRR_lo',
  'CornC4_MissppRS_RFD_hi',
  'CornC4_MissppRS_RFD_lo',
  'CornC4_UsaColoRN',
  'CornC4_UsaColoRN_IRR_hi',
  'CornC4_UsaColoRN_IRR_lo',
  'CornC4_UsaColoRN_RFD_hi',
  'CornC4_UsaColoRN_RFD_lo',
  'CornC4_UsaColoRS',
  'CornC4_UsaCo

Sacar los datos asociados a las variables

In [6]:
region_timeseries = get_xml_etree._1_extract_variable_timeseries(file,region_variables)

Transformar a Data Frame

In [8]:
df = get_xml_etree._1_flatten_region_timeseries(region_timeseries)

In [ ]:
df_pivot = get_xml_etree.tranformDataFrame(df)

### Data Frame total

In [4]:
import os
os.chdir('../output')
import xml2py 
import pandas as pd
import pickle as pkl

In [ ]:
data = xml2py.get_data_threaded(file = file, max_workers=os.cpu_count() - 1)

In [ ]:
dataFrames_list = []
os.chdir('C:/Users/ignacio.delatorre/Documents/Understanding GCAM/gcam-v8.2-Windows-Release-Package/input/gcamdata/xml')
archivos = [i for i in os.listdir() if "USA" not in i]
n_archivos = len(archivos)
n = 0
for file in os.listdir():
    porc = n/n_archivos
    print(f"\r Procesados {n} archivos de {n_archivos}({porc*100:.2f}%)", end="", flush=True)
    if file.endswith('.xml') and "USA" not in file:
        data = xml2py.get_data_threaded(file = file, max_workers=os.cpu_count() - 1)
        df = xml2py._1_flatten_region_timeseries(data)
        df_pivot = xml2py.tranformDataFrame(df)
        dataFrames_list.append(df_pivot)
    n += 1


 Procesados 2 archivos de 244(0.82%)

KeyError: 'variable'

In [ ]:
with open("datos_input.pkl", "wb") as archivo:
    pkl.dump(data, archivo)

In [3]:
file

'ag_Fert_IRR_MGMT.xml'

In [4]:
data

{'USA': {},
 'Africa_Eastern': {},
 'Africa_Northern': {},
 'Africa_Southern': {},
 'Africa_Western': {},
 'Australia_NZ': {},
 'Brazil': {},
 'Canada': {},
 'Central America and Caribbean': {},
 'Central Asia': {},
 'China': {},
 'EU-12': {},
 'EU-15': {},
 'Ukraine': {},
 'Europe_Non_EU': {},
 'European Free Trade Association': {},
 'India': {},
 'Indonesia': {},
 'Japan': {},
 'Mexico': {},
 'Middle East': {},
 'Pakistan': {},
 'Russia': {},
 'South Africa': {},
 'South America_Northern': {},
 'South America_Southern': {},
 'South Asia': {},
 'South Korea': {},
 'Southeast Asia': {},
 'Taiwan': {},
 'Argentina': {},
 'Colombia': {}}

# Borrador

In [ ]:
from lxml import etree
import os

In [4]:
def get_root (file):
    file_abs = os.path.abspath(file)
    parser = etree.XMLParser(huge_tree=True)
    with open(file_abs, 'rb') as f:
        tree = etree.parse(f, parser)
    root = tree.getroot()
    return root


def is_parameter_node(node):
    if "logit" in node.tag.lower():
        return True
    if (node.text and node.text.strip()) and len(node) == 0:
        return True
    return False


def is_real_variable(node):
    if node.tag in ["region", "LandAllocatorRoot", "scenario"]:
        return False
    if node.attrib.get("name") is None:
        return False
    return True

def process_node_logic(node, parent_variable=None, region_dict=None):
    if is_real_variable(node):
        var_name = node.attrib["name"]
        if region_dict is not None and var_name not in region_dict:
            region_dict[var_name] = []
        parent_variable = var_name

    has_parameter = False
    for child in node:
        if parent_variable and region_dict is not None and is_parameter_node(child):
            region_dict[parent_variable].append(child.tag)
            has_parameter = True
        # Procesa recursivamente los hijos sin bucles redundantes
        process_node_logic(child, parent_variable, region_dict)

def extract_variable_timeseries(var_node):
    """
    Extrae series temporales (variables con atributo 'year') dentro de un nodo de variable.
    """
    timeseries = {}

    # Recorremos todos los subnodos recursivamente
    for elem in var_node.iter():
        if "year" in elem.attrib:
            tag = elem.tag
            text = elem.text.strip() if elem.text else None
            if text:
                try:
                    value = float(text)
                except ValueError:
                    continue  # si no es numérico, ignoramos

                year = int(elem.attrib["year"])
                if tag not in timeseries:
                    timeseries[tag] = []
                timeseries[tag].append({"year": year, "value": value})

    # Ordenamos las listas por año
    for tag in timeseries:
        timeseries[tag] = sorted(timeseries[tag], key=lambda x: x["year"])

    return timeseries       

def process_region_xml(xml_str):
    from lxml import etree
    region_node = etree.fromstring(xml_str)
    region_name = region_node.attrib.get("name", "UnknownRegion")
    
    variables = {}
    process_node_logic(region_node, parent_variable=None, region_dict=variables)
    variables_list = list(variables.keys())
    
    region_data = {}
    for var_name in variables_list:
        var_node = region_node.find(f".//*[@name='{var_name}']")
        if var_node is not None:
            ts = extract_variable_timeseries(var_node)
            if ts:
                region_data[var_name] = ts
    return region_name, region_data


def get_data_parallel(file, n_processes=None):
    root = get_root(file)
    region_nodes = root.findall(".//region")
    
    # Convertimos cada nodo a string para poder enviarlo a otros procesos
    region_xmls = [etree.tostring(node) for node in region_nodes]
    
    region_timeseries = {}
    from multiprocessing import Pool
    with Pool(processes=n_processes) as pool:
        results = pool.map(process_region_xml, region_xmls)
    
    for region_name, data in results:
        region_timeseries[region_name] = data
        
    return region_timeseries

def get_data(file):
    root = get_root(file)
    # Recorrer regiones
    #region_variables = {}
    region_timeseries = {}
    for region_node in root.findall(".//region"):
        region_name = region_node.attrib.get("name", "UnknownRegion")
        variables = {}  # variables de esta región
        process_node_logic(region_node, parent_variable=None, region_dict=variables)
        variables_list = list(variables.keys())
        region_timeseries[region_name] = {}
        #region_variables[region_name] = variables_list
        for var_name in variables_list:
            var_node = region_node.find(f".//*[@name='{var_name}']")
            if var_node is not None:
                ts = extract_variable_timeseries(var_node)
                if ts:
                    region_timeseries[region_name][var_name] = ts
        
    return region_timeseries

In [5]:
from concurrent.futures import ThreadPoolExecutor

from timeseries_cy import extract_variable_timeseries_cy

# reemplazas


def get_data_threaded(file, max_workers=None):
    root = get_root(file)
    region_nodes = root.findall(".//region")
    region_timeseries = {}

    def worker(node):
        region_name = node.attrib.get("name", "UnknownRegion")
        variables = {}
        process_node_logic(node, parent_variable=None, region_dict=variables)
        variables_list = list(variables.keys())
        region_data = {}
        for var_name in variables_list:
            var_node = node.find(f".//*[@name='{var_name}']")
            if var_node is not None:
                ts = extract_variable_timeseries_cy(var_node)
                if ts:
                    region_data[var_name] = ts
        return region_name, region_data

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for region_name, data in executor.map(worker, region_nodes):
            region_timeseries[region_name] = data

    return region_timeseries


In [6]:
aaa = get_data_threaded(file, max_workers=None)

In [20]:
aaa = get_data_threaded(file, max_workers=None)

In [ ]:
aaa = get_data_parallel(file)

In [19]:
aaa = get_data(file)

In [305]:
aaa

{'USA': {'AgroForestLand_PacArctic': {'logit-exponent': [{'year': 1975,
     'value': 0.75},
    {'year': 1975, 'value': 2.0}],
   'allocation': [{'year': 1700, 'value': 0.0},
    {'year': 1700, 'value': 0.0},
    {'year': 1750, 'value': 0.0},
    {'year': 1750, 'value': 0.0},
    {'year': 1800, 'value': 3.68e-05},
    {'year': 1800, 'value': 1.32e-05},
    {'year': 1850, 'value': 0.000419},
    {'year': 1850, 'value': 0.000151},
    {'year': 1900, 'value': 0.0012423},
    {'year': 1900, 'value': 0.0004477},
    {'year': 1950, 'value': 0.0046825},
    {'year': 1950, 'value': 0.0016875},
    {'year': 1975, 'value': 0.0046257},
    {'year': 1975, 'value': 0.0018643}],
   'landAllocation': [{'year': 1975, 'value': 0.0046257},
    {'year': 1975, 'value': 0.0018643},
    {'year': 1990, 'value': 0.0060049},
    {'year': 1990, 'value': 0.0004551},
    {'year': 2005, 'value': 0.006095},
    {'year': 2005, 'value': 0.000165},
    {'year': 2010, 'value': 0.0058728},
    {'year': 2010, 'value': 0

In [283]:
def is_parameter_node(node):
    if "logit" in node.tag.lower():
        return True
    if (node.text and node.text.strip()) and len(node) == 0:
        return True
    return False


def is_real_variable(node):
    if node.tag in ["region", "LandAllocatorRoot", "scenario"]:
        return False
    if node.attrib.get("name") is None:
        return False
    return True


def process_node_logic(node, parent_variable=None, region_dict=None):
    if is_real_variable(node):
        var_name = node.attrib["name"]
        if region_dict is not None and var_name not in region_dict:
            region_dict[var_name] = []
        parent_variable = var_name

    has_parameter = False
    for child in node:
        if parent_variable and region_dict is not None and is_parameter_node(child):
            region_dict[parent_variable].append(child.tag)
            has_parameter = True
        # Procesa recursivamente los hijos sin bucles redundantes
        process_node_logic(child, parent_variable, region_dict)

    return has_parameter

def get_root (file):
    file_abs = os.path.abspath(file)
    parser = etree.XMLParser(huge_tree=True)
    with open(file_abs, 'rb') as f:
        tree = etree.parse(f, parser)
    root = tree.getroot()
    return root



def get_vars(root):
    # Recorrer regiones
    region_variables = {}
    for region in root.findall(".//region"):
        region_name = region.attrib.get("name", "UnknownRegion")
        variables = {}  # variables de esta región
        process_node_logic(region, parent_variable=None, region_dict=variables)
        # Guardar solo nombres de variables, ignorando parámetros
        region_variables[region_name] = list(variables.keys())
    return region_variables
    
    
def extract_variable_timeseries(var_node):
    """
    Extrae series temporales (variables con atributo 'year') dentro de un nodo de variable.
    """
    timeseries = {}

    # Recorremos todos los subnodos recursivamente
    for elem in var_node.iter():
        if "year" in elem.attrib:
            tag = elem.tag
            text = elem.text.strip() if elem.text else None
            if text:
                try:
                    value = float(text)
                except ValueError:
                    continue  # si no es numérico, ignoramos

                year = int(elem.attrib["year"])
                if tag not in timeseries:
                    timeseries[tag] = []
                timeseries[tag].append({"year": year, "value": value})

    # Ordenamos las listas por año
    for tag in timeseries:
        timeseries[tag] = sorted(timeseries[tag], key=lambda x: x["year"])

    return timeseries

def get_data(file):
    root = get_root (file)
    region_variables = get_vars(root)
    # Extraer series temporales por región y variable
    region_timeseries = {}

    for region_name, variables_list in region_variables.items():
        region_timeseries[region_name] = {}

        # Encontrar nodo de región
        region_node = root.find(f".//region[@name='{region_name}']")
        if region_node is None:
            continue

        # Iterar sobre las variables
        for var_name in variables_list:
            var_node = region_node.find(f".//*[@name='{var_name}']")
            if var_node is not None:
                ts = extract_variable_timeseries(var_node)
                if ts:
                    region_timeseries[region_name][var_name] = ts
    return region_timeseries

In [284]:
region_timeseries = get_data(file)

In [286]:
file

'../gcam-v8.2-Windows-Release-Package/input/gcamdata/xml/land_input_2.xml'

In [287]:
root

<Element 'scenario' at 0x00000253427361B0>